<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/LSTM/GeracaoAutomaticaDeTexto_Portugues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geração Automática de Texto com LSTMs - Português

As Redes Neurais Recorrentes também podem ser usadas como modelos generativos. Isso significa que além de serem usadas como modelos preditivos (fazendo previsões), elas podem aprender as sequências de um problema e, em seguida, gerar sequências plausíveis inteiramente novas para o domínio do problema. Modelos Generativos como este são úteis não apenas para estudar o quão bem um modelo aprendeu um problema, mas para saber mais sobre o próprio domínio do problema. 

Uma boa forma de praticar a criação de texto, é usando livros clássicos, os quais já temos uma boa ideia sobre a história e que não estejamos violando nenhum direito de copyright. Muitos livros clássicos já não possuem mais restrição de uso e podem ser usados gratuitamente. Um bom lugar para encontrar esses livros é no site do Projeto Gutenberg. É de lá que usaremos o livro para o qual criaremos um modelo generativo: Alice no País das Maravilhas ou o nome em inglês Alice's Adventures in Wonderland. O arquivo txt do livro pode ser baixado aqui: https://www.gutenberg.org/ebooks/11. 
Este livro tem cerca de 3.300 linhas de texto. O cabeçalho e a marca de final de arquivo foram removidos, já que não são necessários para o que vamos fazer.

Vamos aprender as dependências entre os caracteres e as probabilidades condicionais de caracteres em sequências para que possamos gerar sequências totalmente novas e originais de caracteres. Esta é uma tarefa divertida e recomendo repetir essas experiências com outros livros do Projeto Gutenberg. Essas experiências não se limitam ao texto, você também pode experimentar com outros dados ASCII, como código fonte de linguagens de programação, documentos marcados em LaTeX, HTML ou Markdown e muito mais. 

Faremos aqui algo muito similar ao que foi feito pelo programador, que escreveu um novo livro de Game ofthrones: http://www.businessinsider.com/ai-just-wrote-the-next-book-of-game-of-thrones-for-us-2017-8

In [0]:
# Imports
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
# Carregamos os dados e convertemos para lowercase 
# Estamos usando aqui arquivo texto no formato ASCII
filename = "O_Alienista.txt" # livro de Machado de Assis
raw_text = open(filename).read()
raw_text = raw_text.lower()

Agora que o livro está carregado, devemos preparar os dados para modelagem. Não podemos modelar os caracteres diretamente, em vez disso, devemos converter os caracteres em números inteiros. Podemos fazer isso facilmente, criando um conjunto de todos os caracteres distintos do livro, então criando um mapa de cada caractere para um único inteiro.

In [0]:
# Criando o mapeamento caracter/inteiro
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
chars

['\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'z',
 '§',
 '°',
 'º',
 'à',
 'á',
 'â',
 'ã',
 'ç',
 'é',
 'ê',
 'í',
 'ò',
 'ó',
 'ô',
 'õ',
 'ú',
 'ü',
 '—',
 '’',
 '“',
 '”']

In [0]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '?': 21,
 'a': 22,
 'b': 23,
 'c': 24,
 'd': 25,
 'e': 26,
 'f': 27,
 'g': 28,
 'h': 29,
 'i': 30,
 'j': 31,
 'l': 32,
 'm': 33,
 'n': 34,
 'o': 35,
 'p': 36,
 'q': 37,
 'r': 38,
 's': 39,
 't': 40,
 'u': 41,
 'v': 42,
 'x': 43,
 'z': 44,
 '§': 45,
 '°': 46,
 'º': 47,
 'à': 48,
 'á': 49,
 'â': 50,
 'ã': 51,
 'ç': 52,
 'é': 53,
 'ê': 54,
 'í': 55,
 'ò': 56,
 'ó': 57,
 'ô': 58,
 'õ': 59,
 'ú': 60,
 'ü': 61,
 '—': 62,
 '’': 63,
 '“': 64,
 '”': 65}

Pode haver alguns caracteres que podemos remover para limpar mais o conjunto de dados que reduzirá o vocabulário e poderá melhorar o processo de modelagem. 

In [0]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: {:,d}".format(n_chars))  
print ("Total Vocab: ", n_vocab)

Total Characters: 99,080
Total Vocab:  66


Podemos ver que o livro tem pouco mais de 99.000 caracteres e que quando convertidos para minúsculas, existem apenas 44 caracteres distintos no vocabulário para a rede aprender, muito mais do que os 26 no alfabeto. Agora, precisamos definir os dados de treinamento para a rede. Existe muita flexibilidade em como você escolhe dividir o texto e expô-lo a rede durante o treino. Aqui dividiremos o texto do livro em subsequências com um comprimento de 100 caracteres, um comprimento arbitrário. Poderíamos facilmente dividir os dados por sentenças e ajustar as sequências mais curtas e truncar as mais longas. Cada padrão de treinamento da rede é composto de 100 passos de tempo (time steps) de um caractere (X) seguido por um caracter de saída (y). Ao criar essas sequências, deslizamos esta janela ao longo de todo o livro um caracter de cada vez, permitindo que cada caracter tenha a chance de ser aprendido a partir dos 100 caracteres que o precederam (exceto os primeiros 100 caracteres, é claro). Por exemplo, se o comprimento da sequência é 5 (para simplificar), os dois primeiros padrões de treinamento seriam os seguintes:

* Palavra: CHAPTER
* CHAPT -> E
* HAPTE -> R

In [0]:
# À medida que dividimos o livro em sequências, convertemos os caracteres em números inteiros usando nossa
# tabela de pesquisa que preparamos anteriormente.
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total de Padrões: ", n_patterns)

Total de Padrões:  98980


Agora que preparamos nossos dados de treinamento, precisamos transformá-lo para que possamos usá-lo com o Keras. Primeiro, devemos transformar a lista de sequências de entrada na forma [amostras, passos de tempo, recursos] esperados por uma rede LSTM. Em seguida, precisamos redimensionar os números inteiros para o intervalo de 0 a 1 para tornar os padrões mais fáceis de aprender pela rede LSTM que usa a função de ativação sigmoide por padrão.

In [0]:
# Reshape de X para [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# Normalização
X = X / float(n_vocab)

Finalmente, precisamos converter os padrões de saída (caracteres únicos convertidos em números inteiros) usando Hot-Encoding. Isto é para que possamos configurar a rede para prever a probabilidade de cada um dos 44 caracteres diferentes no vocabulário (uma representação mais fácil) em vez de tentar forçá-lo a prever com precisão o próximo caracter. Cada valor de y é convertido em um vetor com um comprimento 66, cheio de zeros, exceto com um 1 na coluna para a letra (inteiro) que o padrão representa. Por exemplo, quando a letra n (valor inteiro 30) tiver sido transformada usando One-Hot Encoding, vai se parecer com isso:

[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
# One-Hot Encoding da variável de saída
y = np_utils.to_categorical(dataY)
y[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [0]:
X.shape

(98980, 100, 1)

In [0]:
# Modelo LSTM com duas camadas de Dropout com 20%
# O tempo de treinamento é bem longo - 
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Não há conjunto de dados de teste. Estamos modelando todo o conjunto de dados de treinamento para aprender a probabilidade de cada caracter em uma sequência. Não estamos interessados nos mais preciso modelo do conjunto de dados de treinamento (Acurácia de Classificação). Este seria um modelo que prevê cada caracter no conjunto de dados de treinamento perfeitamente. Em vez disso, estamos interessados em uma generalização do conjunto de dados que minimiza a função de perda escolhida. Estamos buscando um equilíbrio entre generalização e
overfitting.

In [0]:
# Define o checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

In [0]:
from datetime import datetime
dt = datetime.now()
for i in range(1000000): i +1
print("tempo de execução", datetime.now() - dt)

tempo de execução 0:00:00.121429


Fit do modelo

In [0]:
from datetime import datetime
dt = datetime.now()
# model.fit(X, y, epochs = 20, batch_size = 128, callbacks = callbacks_list)
model.fit(X, y, epochs = 50, batch_size = 64, callbacks = callbacks_list)
print("tempo de execução", datetime.now() - dt)

Epoch 1/50
98980/98980 [==============================] - 653s - loss: 2.9924   
Epoch 2/50
98980/98980 [==============================] - 653s - loss: 2.8144   
Epoch 3/50
98980/98980 [==============================] - 653s - loss: 2.6979   
Epoch 4/50
98980/98980 [==============================] - 655s - loss: 2.5827   
Epoch 5/50
98980/98980 [==============================] - 747s - loss: 2.4802   
Epoch 6/50
98980/98980 [==============================] - 675s - loss: 2.3980   
Epoch 7/50
98980/98980 [==============================] - 808s - loss: 2.3278   
Epoch 8/50
98980/98980 [==============================] - 862s - loss: 2.2672   
Epoch 9/50
98980/98980 [==============================] - 650s - loss: 2.2152   
Epoch 10/50
98980/98980 [==============================] - 650s - loss: 2.1669   
Epoch 11/50
98980/98980 [==============================] - 651s - loss: 2.1193   
Epoch 12/50
98980/98980 [==============================] - 657s - loss: 2.0817   
Epoch 13/50
98980/98980 [

98980/98980 [==============================] - 651s - loss: 1.6260   
Epoch 34/50
98980/98980 [==============================] - 651s - loss: 1.6108   
Epoch 35/50
98980/98980 [==============================] - 653s - loss: 1.6012   
Epoch 36/50
98980/98980 [==============================] - 656s - loss: 1.5887   
Epoch 37/50
98980/98980 [==============================] - 655s - loss: 1.5805   
Epoch 38/50
98980/98980 [==============================] - 662s - loss: 1.5693   
Epoch 39/50
98980/98980 [==============================] - 657s - loss: 1.5594   
Epoch 40/50
98980/98980 [==============================] - 652s - loss: 1.5488   
Epoch 41/50
98980/98980 [==============================] - 651s - loss: 1.5395   
Epoch 42/50
98980/98980 [==============================] - 652s - loss: 1.5283   
Epoch 43/50
98980/98980 [==============================] - 656s - loss: 1.5234   
Epoch 44/50
98980/98980 [==============================] - 657s - loss: 1.5113   
Epoch 45/50
98980/98980 [===

### Continuar treinando o modelo a partir do checkpoint

In [0]:
# Continuar treinando o modelo a partir do checkpoint

# Carrega os melhores pesos da rede e compila o modelo
filename = 'weights/' +  "weights-improvement-07-1.3928.hdf5-Portuguese"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5-Portuguese"

checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

from datetime import datetime
dt = datetime.now()
# fit the model
model.fit(X, y, epochs = 10, batch_size = 64, callbacks = callbacks_list)
print("tempo de execução", datetime.now() - dt)

Epoch 1/10
98980/98980 [==============================] - 651s - loss: 1.4153   
Epoch 2/10
98980/98980 [==============================] - 650s - loss: 1.4023   
Epoch 3/10
98980/98980 [==============================] - 651s - loss: 1.4026   
Epoch 4/10
98980/98980 [==============================] - 652s - loss: 1.3915   
Epoch 5/10
98980/98980 [==============================] - 650s - loss: 1.3881   
Epoch 6/10
98980/98980 [==============================] - 664s - loss: 1.3892   
Epoch 7/10
98980/98980 [==============================] - 653s - loss: 1.3770   
Epoch 8/10
98980/98980 [==============================] - 658s - loss: 1.3816   
Epoch 9/10
98980/98980 [==============================] - 654s - loss: 1.3758   
Epoch 10/10
98980/98980 [==============================] - 651s - loss: 4.3375   
tempo de execução 1:49:00.916299


In [0]:
# tempo de execução 9:14:18 - 50 epoch
# tempo de execução 1:50:49 - 10 epoch
#!ls w*.-Portuguese
!ls  w*.hdf5-Portuguese

weights-improvement-00-1.4153.hdf5-Portuguese
weights-improvement-01-1.4023.hdf5-Portuguese
weights-improvement-03-1.3915.hdf5-Portuguese
weights-improvement-04-1.3881.hdf5-Portuguese
weights-improvement-06-1.3770.hdf5-Portuguese
weights-improvement-08-1.3758.hdf5-Portuguese


In [0]:
#!ls weights

In [0]:
#!cp weights-improvement-48-1.4750.hdf5 weights-improvement-48-1.4750.hdf5-Portuguese

Depois de executar o fit, você deve ter uma série de arquivos de checkpoint no mesmo diretório onde está este Jupyter Notebook. Você pode excluí-los todos exceto aquele com o menor valor de perda. Por exemplo, neste caso, o arquivo weights-improvement-19-1.9119.hdf5 será usado. Ele contém os melhores valores de peso.

In [0]:
# Carrega os melhores pesos da rede e compila o modelo
filename = "weights-improvement-08-1.3758.hdf5-Portuguese"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
def generate_text():
    # Obtém um random seed
    start = numpy.random.randint(0, len(dataX)-1)

    # Inicia a geração de texto de um ponto qualquer, definido pelo random seed "start"
    pattern = dataX[start]
    print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

    # Gerando caracteres
    for i in range(1000):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print('\n=======================================================================================================\n')

    #print ("\nConcluído.")

In [0]:
generate_text()

"  da câmara, e da matriz;—ou por meio de matraca. 
7
eis em que consistia este segundo uso. contratav "
a-se o dasiica eosa de eizer que o alienista não o alienista foi o pue prdia ser ee uma peruersiria do alienista eos almdossos de poder a sua majestade. a puem entiruava a casa verde e de terris lais esa uma coisa mais de manhão com a ele, não a de janhia, e alnda o alienista puoiaio que vossa renhora e dessibado a putra de ser emrredo de casa espicado e desiguarses e conseger ao povo eoses de mongems, e deserninhar a parse a casa verde é um col de longin e desebrar aos outros, e a alia do res de janeiro. sem de uma peruecaram as alianiste. o povo ene nais a meio destribada d de todas as alianistos de simão bacamarte que o alienista fosse con erande a almga e aligos dos outros, o alienista não o alienista foi o pue prdie fogarda do inuerno com a responta de sua majestade. um dos prineiras, a casa verde é um dos pontos para o ponferoo que o alienista fosse con o alienista fosse conti

In [0]:
for i in range(5):
    generate_text()

" u esse grito
justamente no dia em que simão bacamarte fizera recolher à casa verde um homem que traz "
ia com o alienista eosse continto e alma aodaça a um alienista de casa esa uma casa mais de uma perueca do boticário. e dizia ele não podia ser rodos os casos de um alienisto era o alienista contintou o alienista que o alienista fosse con erande a almga e aligos dos outros, o alienista não o alienista foi o pue prdie fogarda do inuerno com a responta de sua majestade. um dos prineiras, a casa verde é um dos pontos para o ponferoo que o alienista fosse con o alienista fosse continto e alma a dessrra da rerponta de itaguaí e asosria e a dessiga de que o alienista eosse continto e alma aodaça a casa verde, o alienista ficou a puem e de um cos mais consrigades mais aluitres de dois monensos da casa verde e de porta movícia de sua majestade. a puem incomtinte a esta dxperiência eo alienista que o alienista fosse con erande a almga e aligos dos outros, o alienista não o alienista foi o pu

In [0]:
generate_text()

"  e que ele não podia deixar na rua um
mentecapto. a última pessoa que intercedeu por ele (porque dep "
ois de casa a mais pidenda de um alcare de sereltou a casa verde era uma casa aorica, e a câmara lhe dasa de alguma casa verde. — a pazão do boticário. e dizer que era um cos damar, e a dâmara lhe deste lajs de eizer que o alienista fosse conceuso do alienista por angar comses a esperiar dos peus enteresses de ser emtregar com a alna, o perso de que a câmara lhe dasa de ser eme a casa verde era uma casa aoricado, mas acmanan-lhe o alienista a alguns senpos, a casa verde era uma casa aoricado, mas acmanan-lhe o alienista a alguns senpos, a casa verde era uma casa aoricado, mas acmanan-lhe o alienista a alguns senpos, a casa verde era uma casa aoricado, mas acmanan-lhe o alienista a alguns senpos, a casa verde era uma casa aoricado, mas acmanan-lhe o alienista a alguns senpos, a casa verde era uma casa aoricado, mas acmanan-lhe o alienista a alguns senpos, a casa verde era uma casa ao